In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas
import pandas as pd
import numpy as np
import gensim
import csv
import ProbData
from ProbModels import *
import pickle
from utilities import *
from sentences import *
from DCS import *
import os
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold
from IPython.display import display

In [33]:
df = pd.read_csv('pcrw_25_', usecols=['f', 'ln_lemma', 'rn_lemma', 'ln_cng', 'rn_cng', '111',
       '112', '113', '121', '122', '123', '131', '132', '133', '211', '212',
       '213', '221', '222', '223', '231', '232', '233', '311', '312', '313',
       '321', '322', '323', '331', '332', '333', '400', '500', '600', '700',
       '800', '900', 'flag', '011', '022', '033'])

df = df.drop(df[np.isnan(df.ln_cng) | np.isnan(df.rn_cng)].index) # Add

df.ln_cng = df.ln_cng.astype(int) # Add
df.rn_cng = df.rn_cng.astype(int) # Add

In [38]:
np.sum(df['033'] > 0)

305205

In [39]:
df.head(6)

,f,ln_lemma,rn_lemma,ln_cng,rn_cng,111,112,113,121,122,...,400,500,600,700,800,900,flag,011,022,033
0,123274.p2,arjuna,vaDa,149,35,0.000223,0.000053,0.0,0,0.000047,...,0.0,0.0,0.0,0.0,0.0,0,1,0.001098,0.000518,0.0
1,123274.p2,arjuna,vaDa,151,35,0.000223,0.000053,0.0,0,0.000047,...,0.0,0.0,0.0,0.0,0.0,0,1,0.001098,0.000410,0.0
2,123274.p2,arjuna,vaDa,149,36,0.000223,0.000055,0.0,0,0.000020,...,0.0,0.0,0.0,0.0,0.0,0,1,0.001098,0.000628,0.0
3,123274.p2,arjuna,vaDa,151,36,0.000223,0.000055,0.0,0,0.000020,...,0.0,0.0,0.0,0.0,0.0,0,1,0.001098,0.000990,0.0
4,123274.p2,arjuna,vaDa,149,75,0.000223,0.000034,0.0,0,0.000128,...,0.0,0.0,0.0,0.0,0.0,0,1,0.001098,0.000130,0.0
5,123274.p2,arjuna,vaDa,151,75,0.000223,0.000034,0.0,0,0.000128,...,0.0,0.0,0.0,0.0,0.0,0,1,0.001098,0.000045,0.0


In [6]:
# pb = ProbModels(fullCo_oc_mat = ProbData.fullCo_oc_mat, unigram_counts = ProbData.unigram_counts,
#                cng2cngFullMat = ProbData.cng2cngFullMat, cng2index_dict = ProbData.cng2index_dict,
#                w2w_samecng_fullmat=ProbData.w2w_samecng_fullmat, samecng_unigram_counts=ProbData.samecng_unigram_counts,
#                v2c_fullMat = ProbData.v2c_fullMat)

In [25]:
# 11 - lemma2lemma
# 22 - cng2cng
# 33 - samecng
# def lemlem(row):
#     try:
# #         return ProbData.fullCo_oc_mat[row['ln_lemma']][row['rn_lemma']] / ProbData.unigram_counts[row['ln_lemma']]
#         return pb.kn_word2word(row['ln_lemma'],row['rn_lemma'])
#     except KeyError:
#         return 0
    
# def lemlem_sc(row):
#     if row['ln_cng'] == row['rn_cng']:
#         try:
# #             return ProbData.w2w_samecng_fullmat[row['ln_lemma']][row['rn_lemma']] / ProbData.samecng_unigram_counts[row['ln_lemma']]
#             return pb.kn_word2word_samecng(row['ln_lemma'],row['rn_lemma'])
#         except KeyError:
#             return 0
#     else:
#         return 0
    
# def cngcng(row):
#     try:
# #         ia = ProbData.cng2index_dict[str(row['ln_cng'])]
# #         ib = ProbData.cng2index_dict[str(row['rn_cng'])]
# #         return ProbData.cng2cngFullMat[ia, ib] / pb.t2t_co_oc_count[ia, 0]
#         return float(pb.kn_cng2cng(row['ln_cng'], row['rn_cng']))
#     except KeyError:
#         return 0
        

In [ ]:
# row_11 = df.apply(lemlem, axis = 1)

# row_22 = df.apply(cngcng, axis = 1)

# row_33 = df.apply(lemlem_sc, axis = 1)

In [ ]:
# i = 0
# for index, row in df.iterrows():
#     i += 1
#     if i == 200:
#         break
#     print(cngcng(row))

In [29]:
# df['011'] = row_11
# df['022'] = row_22
# df['033'] = row_33

In [40]:
# hList = ['f', 'ln_lemma', 'rn_lemma', 'ln_cng', 'rn_cng']
hList = []
for u in range(1,10):
    if u < 4:
        for v in [1,2,3]:
            for w in [1,2,3]:
                hList.append(u*100 + 10*v + w)
    else:
        hList.append(u*100)
# hList.extend(['011','022','033'])
hList.extend(['011','022', '033'])

# hList.append('flag')

goodCols = []
for h in hList:
    if np.min(df[str(h)]) == 0 and np.max(df[str(h)]) == 0:
#         print(h)
        pass
    else:
        goodCols.append(str(h))
        
print(goodCols)

['111', '112', '113', '122', '123', '131', '132', '211', '212', '213', '222', '223', '231', '232', '311', '312', '322', '400', '500', '600', '700', '800', '011', '022', '033']


In [41]:
len(goodCols)

25

In [42]:
df.flag.value_counts()

0    4283777
1     801548
Name: flag, dtype: int64

In [43]:
n_indices = list(range(4300000))
n_indices = n_indices[0:4004008:5]
print(len(n_indices), max(n_indices))

800802 4004005


In [44]:
df_n = df[df['flag'] == 0]
df_p = df[df['flag'] == 1]

df_n = df_n.sample(df_p.shape[0])
print(df_n.shape, df_p.shape)

(801548, 42) (801548, 42)


In [45]:
frames = [df_n, df_p]
df_2 = pd.concat(frames)
df_2 = df_2.sample(df_2.shape[0])

In [46]:
print(df_2[df_2['flag'] == 0].shape)
print(df_2[df_2['flag'] == 1].shape)

(801548, 42)
(801548, 42)


In [47]:
df_2 = df_2.reset_index(drop=True)

In [48]:
lr = LogisticRegression(solver='lbfgs', fit_intercept=False) # lbfgs, No Bias
# lr.fit()
kf = KFold(df_2.shape[0],4)

In [49]:
coeffs = []
for train, test in kf:
    lr.fit(df_2.loc[train, goodCols], df_2.loc[train, 'flag'])
    print(np.sum(df_2.loc[test, 'flag'] == lr.predict(df_2.loc[test, goodCols]))/len(test))
    coeffs.append(lr.coef_)

0.695047582927
0.695599015904
0.695055068443
0.693802492178


In [50]:
df_results = pd.DataFrame(columns=['Path', 'CV_1', 'CV_2' ,'CV_3', 'CV_4'])
for i in range(len(goodCols)):
    df_results.loc[i] = [goodCols[i], coeffs[0][0][i], coeffs[1][0][i], coeffs[2][0][i], coeffs[3][0][i]]

In [51]:
df_results['avg'] = np.sum(df_results[['CV_1', 'CV_2', 'CV_3', 'CV_4']], axis = 1)/4
df_results['min'] = np.min(df_results[['CV_1', 'CV_2', 'CV_3', 'CV_4']], axis = 1)

In [52]:
print(goodCols)
display(np.asarray(df_results['avg']))

['111', '112', '113', '122', '123', '131', '132', '211', '212', '213', '222', '223', '231', '232', '311', '312', '322', '400', '500', '600', '700', '800', '011', '022', '033']


array([ -2.30184849,   0.15205544,  11.37284794,   2.22866016,
         2.5926891 ,  27.34128227,  -2.72856657, -15.95316347,
        -4.30608648,   9.95769945,  -3.72684989,   5.79208033,
         2.16235142,  13.69485912,   6.77834282,   0.43567416,
         1.20300337,   2.95172869,   0.24279471,   0.30121553,
         1.29004227,   0.27179539,  30.28146594,  -0.21266843,   9.93318708])

In [59]:
# print(goodCols)
# display(np.asarray(df_results['avg']))

['111', '112', '113', '122', '123', '131', '132', '211', '212', '213', '222', '223', '231', '232', '311', '312', '322', '400', '500', '600', '700', '800', '011', '022', '033']


array([ -3.60154202,   0.12051034,  10.20001181,   2.19923501,
         2.28734689,  25.36362492,  -3.09226718, -15.77969152,
        -4.21303213,  10.62164022,  -3.71344374,   5.8242817 ,
         2.13084679,  13.42347203,   6.94626147,   0.33661488,
         1.22412063,   2.88330069,   0.20301016,   0.32069797,
         1.04363305,   0.28507474,   4.49850795,  -0.27397026,   8.43255253])

In [40]:
np.asarray(df_results.loc[df_results['min'] > 0, 'avg'])

array([  0.27920666,  11.28729653,   2.17634043,   2.94292211,
        27.6274655 ,  10.75837206,   5.45059613,   2.12671972,
        13.65112367,   7.28889777,   0.37595928,   1.2250849 ,
         2.95362505,   0.23630001,   0.33619974,   1.1608632 ,
         0.29997654,   4.66070081])

In [41]:
df_results

,Path,CV_1,CV_2,CV_3,CV_4,avg,min
0,111,-4.782406,-5.169520,-4.100222,-4.389363,-4.610378,-5.169520
1,112,0.233140,0.253134,0.243293,0.387258,0.279207,0.233140
2,113,10.933322,11.916028,11.497238,10.802599,11.287297,10.802599
3,122,2.202385,2.152021,2.181509,2.169447,2.176340,2.152021
4,123,2.832715,2.667016,2.985158,3.286798,2.942922,2.667016
5,131,26.794290,28.637952,27.345118,27.732503,27.627465,26.794290
6,132,-3.662640,-3.737598,-3.042336,-3.557660,-3.500059,-3.737598
7,211,-16.284828,-15.409822,-16.406734,-16.044439,-16.036456,-16.406734
8,212,-4.034703,-4.189716,-4.225610,-4.365946,-4.203994,-4.365946
9,213,10.584532,11.152043,10.704669,10.592245,10.758372,10.584532


In [18]:
def getname(c):
    if c == '1':
        return 'simple lemma'
    if c == '2':
        return 'simple cng'
    if c == '3':
        return 'simple word'
    if c == '4':
        return 'noun-noun'
    if c == '5':
        return 'verb-gerund'
    if c == '6':
        return 'verb-absol'
    if c == '7':
        return 'verb-ppp'
    if c == '8':
        return 'verb-inf'
    if c == '9':
        return 'verb-ppa'
df_path = pd.DataFrame(columns=['Path Code', 'Element 1', 'Element 2', 'Element 3'])
for i in range(len(goodCols)):
    path = goodCols[i]
    df_path.loc[i] = [path, getname(path[0]), getname(path[1]), getname(path[2])]

In [19]:
df_path

,Path Code,Element 1,Element 2,Element 3
0,111,simple lemma,simple lemma,simple lemma
1,112,simple lemma,simple lemma,simple cng
2,113,simple lemma,simple lemma,simple word
3,122,simple lemma,simple cng,simple cng
4,123,simple lemma,simple cng,simple word
5,131,simple lemma,simple word,simple lemma
6,132,simple lemma,simple word,simple cng
7,211,simple cng,simple lemma,simple lemma
8,212,simple cng,simple lemma,simple cng
9,213,simple cng,simple lemma,simple word


In [35]:
goodWeights = np.array([0.30758324,  12.0556656 ,   2.17109444,   5.78959942, 33.49313423,  10.32264846,   7.94864612,   4.31576204, 13.58834529,   9.17713191,   0.24156488,   1.22543121, 3.23680681,   0.33608101,   0.39159008,   1.72731087,   0.36444531])


In [36]:
goodWeights = goodWeights/np.sum(goodWeights)

In [37]:
goodWeights

array([ 0.00288289,  0.11299414,  0.02034902,  0.05426418,  0.3139211 ,
        0.09675109,  0.07450028,  0.04045034,  0.12735949,  0.08601451,
        0.00226412,  0.0114856 ,  0.03033762,  0.00314999,  0.00367026,
        0.01618957,  0.00341584])

In [3]:
w3 = np.array([0.3, 0.1, 0.55])
w3 *= 0.4

In [4]:
w3

array([ 0.12,  0.04,  0.22])

In [40]:
np.sum(goodWeights*0.6) + np.sum(w3)

0.97999999999999998

In [41]:
np.sum(w3)

0.38

In [3]:
goodCols = ['111', '112', '113', '122', '123', '131', '132', '211', '212', '213', '222', '223', '231', '232', '311', '312', '322', '400', '500', '600', '700', '800']

In [23]:
x = np.asarray(df_results['avg'])

In [24]:
z = x/(np.max(x) - np.min(x))

In [25]:
z

array([-0.06366967,  0.00180837,  0.23795702,  0.0506419 ,  0.04758066,
        0.62008073, -0.07183627, -0.37991927, -0.09453072,  0.22684876,
       -0.08394839,  0.13554915,  0.05121899,  0.3129761 ,  0.15254854,
        0.00922738,  0.02676178,  0.06565262,  0.00351618,  0.00659052,
        0.02472485,  0.00597698,  0.10132104, -0.00659054,  0.19438582])

In [42]:
print(goodCols)
print(np.asarray(df_results['avg']))

['111', '112', '113', '122', '123', '131', '132', '211', '212', '213', '222', '223', '231', '232', '311', '312', '322', '400', '500', '600', '700', '800', '011', '022']
[ -4.61037792   0.27920666  11.28729653   2.17634043   2.94292211
  27.6274655   -3.50005862 -16.03645576  -4.20399383  10.75837206
  -3.65632308   5.45059613   2.12671972  13.65112367   7.28889777
   0.37595928   1.2250849    2.95362505   0.23630001   0.33619974
   1.1608632    0.29997654   4.66070081  -0.42086444]


In [30]:
df.to_csv('pcrw_25_smooth.csv')

In [57]:
df_results

,f,ln_lemma,rn_lemma,ln_cng,rn_cng,111,112,113,121,122,...,331,332,333,400,500,600,700,800,900,flag
0,123274.p2,arjuna,vaDa,149,35,0.000223,0.000053,0.0,0,0.000047,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0,1
1,123274.p2,arjuna,vaDa,151,35,0.000223,0.000053,0.0,0,0.000047,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0,1
2,123274.p2,arjuna,vaDa,149,36,0.000223,0.000055,0.0,0,0.000020,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0,1
3,123274.p2,arjuna,vaDa,151,36,0.000223,0.000055,0.0,0,0.000020,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0,1
4,123274.p2,arjuna,vaDa,149,75,0.000223,0.000034,0.0,0,0.000128,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0,1
